<div style="text-align: right">INFO 6105 Data Science Eng Methods and Tools, Midterm prep</div>
<div style="text-align: right">Dino Konstantopoulos, 24 February 2020</div>

# Generating random numbers with arbitrary distribution

<br />
<center>
<img src="https://media3.giphy.com/media/UReWF9frq7Rv7ZIqhy/giphy.gif" width=400 />
</center>

The reason why we use well-known model functions (like the normal or the gamma) in statistics is because of the ***huge*** dimensionality reduction when you use an analytic function: All you need to find out are its parameters, which is usually one, two, or three!

Nevertheless, with a little bit of coding, you actually *don't need any math at all* (what I told you at the beginning of the semester), you can ***build a model from the data without any math, with a bit of programming***. Either you *know the math*, or you *write the code*! In this notebook, we'll write code for our model instead of doing the math. This results in a not-so-dramatic dimensionality reduction, but it's still a general model that you can reuse.

## 1. Warm-up

Let's generate random variates from a normal distribution, then plot the histogram of the data. You *always* start with a nomral distribution in statistics!

In [ ]:
import scipy.stats
import numpy as np
data = scipy.stats.norm.rvs(size=100000, loc=0, scale=1.5, random_state=123)

Now compute the histogram (the frequency of all the values), and pick the number of bins you want to divide the values in:
```(python)
hist = np.histogram(data, bins=?)
```

Now let's plot the data and the histogram:

In [ ]:
len(data)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.title("data with normal distribution")
plt.plot(data[0:1000], label='data')
plt.show()

In [ ]:
hist

In [ ]:
plt.title("histogram of data with normal distribution")
plt.plot(hist[0], label='histogram')
plt.show()

what is `hist[1]`?

The answer to your homework is a single API call:

`scipy.stats.rv_histogram` is a neat API: It produces a pdf from a histogram.

> Huh?! Yup, `scipy.stats.rv_histogram` is essentially the answer to your homework. A single API call! If you only knew your libraries, imagine how much GoT you could watch if you knew all the APIs that exist!

In [ ]:
hist_dist = scipy.stats.rv_histogram(hist)

`hist_dist` behaves like an ordinary scipy rv_continuous distribution. For example, we can obtain its pdf (probability of obtaining a value) and cdf (probability of obtaining any value *below* a specific value):

In [ ]:
hist_dist.pdf(1.0)

In [ ]:
 hist_dist.cdf(0.0)

PDF is zero after the last bin of the histogram, and also before the first bin of the histogram, defined by the max and min of the original dataset:

In [ ]:
print(hist_dist.pdf(np.max(data)))
print(hist_dist.cdf(np.max(data)))
print(hist_dist.pdf(np.min(data)))
print(hist_dist.cdf(np.min(data)))

Let's plot the PDF ***on top of*** the histogram we obtained from our data to see if it matches:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
X = np.linspace(-5.0, 5.0, 100)
plt.title("histogram and PDF from Template")
plt.hist(data, density=True, bins=100)
plt.plot(X, hist_dist.pdf(X), label='PDF')
plt.show()

Now let's also plot the CDF on top of the histogram we obtained from our data:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
X = np.linspace(-5.0, 5.0, 100)
plt.title("CDF from Template")
plt.hist(data, density=True, bins=100)
plt.plot(X, hist_dist.cdf(X), label='CDF')
plt.show()

Notice how the ***0.5 value of the cdf*** is right above the mean of your histogram! The probability of getting *exactly* the mean value of the dataset is exactly 50% because our histogram is balanced and not *skewed* to the right or the left: Same amount of data to the right and to the left of the histogram. 

We'll revisit this fact further below in the notebook, so keep this in mind.

## 2. Intuition

To really do your homework, what you need to create is a **lookup table**. The histogram gives us bins of frequency. When we generate random variates, we lookup those bins *with priority proportional to how big they are*. 

The intuition comes from genetic algorithms' (GA) roulette wheel, a.k.a. [fitness-proportionate selection](https://en.wikipedia.org/wiki/Fitness_proportionate_selection), as we will see further down.

Anyway, this is our *input* histogram:

In [ ]:
hist

In [ ]:
hist[0]

Here is my *first cut* at the function: The idea was to create a number of workers, and to have workers work an amount of time proportional to the histogram. Then we interrupt the coroutine that represents their work (by yielding) and see which worker was active. The lazy workers (short bins) are already done and are probably sleeping, while the very active workers (tall bins) are still working. Those are the workers (bins) I want to pick values from!

<br />
<center>
<img src="https://media0.giphy.com/media/oDrajdEXgcAvK/source.gif" width=400 />
    Working, working, working...
</center>

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('HI0x0KYChq4')

And as soon as I finished writing this, I thought *this is exactly GA roulette wheel selection*! Here it is, as simple as possible:

In [ ]:
hsgm = hist[0]

class Chromosome:
    def __init__(self, idx, fit):
        self.fitness = fit
        self.index = idx
        
Chromosomes = []

# this is ugly python
#i = 0
#for _ in hsgm:
#    Chromosomes.append(Chromosome(i, _))
#    i += 1

# this is pretty python
for i, _ in enumerate(hsgm):
    Chromosomes.append(Chromosome(i, _))

What's the index of the 9$^{th}$ bin, and what is its size? Write the code below:

Now, how to randomly pick a bin so that most of the time you pick a tall bin? Looat the API of [np.random.choice](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.choice.html) and write the code here below to return a Chromosome from a tall bin more often than one from a short bin: 

>**HINT**: First renormalize chromosomes (using a list comprehension) so that their fitness becomes a probability distriubution (so, sums to 1), then use [np.random.choice](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.choice.html) to return a chromosome from a tall bin since you can now pass the API a probability distribution.

<div style="visibility: hidden">
def selectOne(population):
    maximum = sum([c.fitness for c in population])
    selection_probs = [c.fitness/maximum for c in population]
    return population[np.random.choice(len(population), p=selection_probs)]
</div>

Now let's test that function and see if it returns what we want:
```(python)
c = selectOne(Chromosomes)
c.index, c.fitness
```

Here is a full algorithm without "*cheating*" by using `np.random.choice`. Copy and paste each code snippet into the code cell further below.

First, sort the weights in ascending order: This gives you indeces and weights based on weights in ascending order: We use python's `sorted` API, follwed by the zipper to get both index and weight (height of the bin):
```(python)
    sorted_indexed_weights = sorted(enumerate(weights), key=operator.itemgetter(1));
    indices, sorted_weights = zip(*sorted_indexed_weights);
```

Then we calculate the cumulative probability (note that is a cdf!). Essentially, we convert  weights to a pdf, then to a cdf. A cdf is useful because the tallest bin in the distribution (the mean) is associated with a cdf value of 0.5. We leverage `Numpy`'s [cumsum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cumsum.html) API
```(python)
    tot_sum = np.sum(sorted_weights)
    prob = [x/tot_sum for x in sorted_weights]
    cum_prob = np.cumsum(prob)
```

Now we select a random a number in the range \[0,1\]:
```(python)
    random_num = random.random()
```

Now we go through the cdf. As soon as we find a bin where the cdf value exceeds our random float, return the index
of that bin:
```(python)
    for index_value, cum_prob_value in zip(indices, cum_prob):
        if random_num < cum_prob_value:
            return index_value
```

In [ ]:
import random
import operator

def roulette_selection(weights):
    '''performs roulette wheel selection on a list, returns the index selected from the list'''


    

    

    
    
    
    
    
    
    
    
    
    

Let's walk through this algorithm block by block:

In [ ]:
hsgm = hist[0]
sorted_indexed_weights = sorted(enumerate(hsgm), key=operator.itemgetter(1))
'; '.join(['('+str(p)+', '+str(q)+')' for (p,q) in sorted_indexed_weights])

In [ ]:
indices, sorted_weights = zip(*sorted_indexed_weights);
', '.join([str(i) for i in indices])

In [ ]:
', '.join([str(i) for i in sorted_weights])

In [ ]:
tot_sum = np.sum(sorted_weights)
prob = [x/tot_sum for x in sorted_weights]
cum_prob = np.cumsum(prob)
cum_prob

In [ ]:
', '.join([str(i) for i in prob])

In [ ]:
', '.join([str(round(i,2)) for i in cum_prob])

Now select a random float in \[0, 1\]:

In [ ]:
random_num = random.random()
random_num

>Why is this algorithm called **roulette wheel** or ***fitness proportionate*** selection? Because if you decided to use a random number generator to select your next boyfriend/girlfriend and somebody told you that you need to include the eventuality of getting a boyfriend/girlfriend that you dont like that much, you would want to use a normal distribution and assign the bins closer to the mean to the prettiest possible girls and boys, right?

That's the level of fitness you ***expect*** (expectation) for your next boyfriend/girlfriend! In the code above, the first entry in the list above which is bigger than a random level of fitness with expectation = 0.5 (expectation of a normal random number generator)! 

<br />
<center>
<img src="https://media3.giphy.com/media/l0HlIAWUuhUccrtDO/giphy.gif" width=300 />
</center>

This is the same thing as spinning a roulette wheel with pieces of the pie assigned proportionately to prettier people and seeing which piece of the pie it lands on after a spin!

<br />
<center>
<img src="https://www.researchgate.net/publication/311245613/figure/fig3/AS:566020097220608@1511961115040/Roulette-wheel-selection-based-on-fitness.png" width=400 />
</center>

What is the expectation for your random number? 0.5, right? You can see that 0.5 lands you right next to the "*big*" bins, the ones with big fitness values.

In [ ]:
for index_value, cum_prob_value in zip(indices, cum_prob):
    if random_num < cum_prob_value:
        print(index_value)
        break

In [ ]:
len(hist[0])

Now run the cell below *many many* times and see if it gives you *most of the time*, indeces close to halh the size of your histogram:

In [ ]:
roulette_selection(hist[0])

The input to your homework function is a histogram $h$ (with any number of bins), a range $r$ to generate values from, and the number $n$ of desired random variates.

Now, break down the input interval into *as many intervals as there are bins in the input histogram*. Then we use roulette wheel selection to pick one of these bins. Then generate a random float within the interval of that bin, and that is our first random variate!

Use the following inputs:
```(python)
# inputs
h = hsgm
r = range(-23, 23)
n = 10000
```

Now subdivide the x axis in the r range into `len(hsgm)` bins by using the `np.inspace` API nd verify that `x[0], x[len(hsgm)//2], x[len(hsgm)-1]` are the values you expected for the range `r`:

Then call `i = roulette_selection(h)` many times and `print(i)`. Also, add the following below (why do we do this?):
```(python)
if i == len(hsgm) - 1: i -= 1
x[i], x[i+1]
```

Now generate random values **uniformly** (using `random.uniform`) API to generate random values from the interval `x[i], x[i+1]`:

<div style="visibility:hidden">
random.uniform(x[i], x[i+1])
</div>

Ready?

Now, generate a list of random variates whose distribution will look like our initial normal distribution `h`:
```python)
my_random_variates = []
for _ in range(n):
    i = roulette_selection(h)
    if i == len(hsgm) - 1: i -= 1
    my_random_variates.append(random.uniform(x[i], x[i+1]))
```

Now plot the random variates:

In [ ]:
plt.title("data with desired distribution")
plt.plot(my_random_variates, label='data')
plt.show()

Now plot the histogram of our random variates and superimpose that on top of our input histogram. Use the following renormalization so that the maximums of both histograms (*input* and *generated*) match:
```(python)
plt.plot(hsgm * max(my_not_so_random_histogram[0]) / max(hsgm), label='original histogram')
```

In [ ]:
my_not_so_random_histogram = np.histogram(my_random_variates, bins=len(hsgm))

In [ ]:
plt.title("my not-so-random histogram")
plt.plot(my_not_so_random_histogram[0], label='histogram')
...
plt.show()

And... what if we generate ten times more datapoints, say 100,000?
```(python)
my_random_variates = []
for _ in range(n * 10):
    i = roulette_selection(h)
    if i == len(hsgm) - 1: i -= 1
    my_random_variates.append(random.uniform(x[i], x[i+1]))
    
my_not_so_random_histogram = np.histogram(my_random_variates, bins=len(hsgm))

plt.title("my not-so-random and also improved histogram")
plt.plot(my_not_so_random_histogram[0], label='generated histogram')
plt.plot(hsgm * max(my_not_so_random_histogram[0]) / max(hsgm), label='original histogram')
plt.show()
```

Getting closer and closer the more random variates I produce! This is the key to *frequentist statistics*. If i have tons of data, then I'm in good shape. However, *when we don't have enough data*, which is the *sweet spot( of state of the art ML, that is when we turn to *Bayesian statistics*!

## Yang Huang's function

This is a student's function, who wondered why we had to do roulette wheel selection:

In [ ]:
import random
## This function takes a histogram 'A'(tuple), size of fakedata 'N'(int) and the range of fakedata 'R' (tuple) as input
def fakeData(A, N, R):
    nOfBins = A[0].size
    sum = 0
    for _ in A[0]:
        sum +=_
    ## n is the size of original dataset
    n = sum 
    
    ## set the frequency in fake dataset
    frequency = []
    for i in range(nOfBins):
        frequency.append(A[0][i]/n*N)
        
    ## set the width of bins in new dataset
    w = (R[1]-R[0])/nOfBins
    
    ## set ranges for new bins
    ranges = []
    for i in range(nOfBins+1):
        ranges.append(R[0] + i*w)
        
    ## create fakedata   
    data = []
    #this is beautiful python
    for m, i in enumerate(frequency):
        for _ in range(int(i)):
            data.append(random.uniform(ranges[m],ranges[m+1]))
    
    return data          

Let's examine what Yang's function does:

In [ ]:
hist[0]

In [ ]:
N = 10000
R = [-23, 23]
nOfBins = hist[0].size
sum = 0
for _ in hist[0]:
    sum +=_
## n is the size of original dataset
n = sum 
n

>**CORRECTION**: n is the **cumsum** (or cumulative sum) of the histogram, ***not*** the size of of the dataset, which is `len(hist[0]) = 100`

In [ ]:
frequency = []
for i in range(nOfBins):
    frequency.append(hist[0][i]/n*N)
frequency

So it appears `frequency` is just a *rescaled* histogram.

In [ ]:
## set the width of bins in new dataset
w = (R[1]-R[0])/nOfBins
w

So we will generate values from intervals of size `w` above, with frequency proportional to the size of the corresponding histogram bin! So far, so good!

In [ ]:
## set ranges for new bins
ranges = []
for i in range(nOfBins+1):
    ranges.append(R[0] + i*w)
ranges

And those are the intervals we will generate values from. Grrrrrrrrrrrreat!

In [ ]:
## create fakedata   
data = []
#this is beautiful python
for m, i in enumerate(frequency):
    for _ in range(int(i)):
        data.append(random.uniform(ranges[m],ranges[m+1]))
data

`for m, i in enumerate(frequency):` essentially goes through the original histogram and gets the bin index `m` and the bin count `i`.

In that outer loop. `for _ in range(int(i)):` ranges the bin count. So it's a much larger range of numbers for big bins (bins with big values), and a much smaller range of numbers for small bins. In other words, what happens under the `for` loop will happen ***more often*** when we're visiting big bins than when we're visiting small bins. In other words, this is **roulette wheel selection**.

In that inner loop, `random.uniform(ranges[m],ranges[m+1])` selects a single random number, with uniform probability between `ranges[m]` and `ranges[m+1]`:

In [ ]:
print(ranges[10],ranges[11])
random.uniform(ranges[10],ranges[11])

Finally, `data.append` appends the value to our list of generated numbers.

<br />
<center>
<img src="https://media1.giphy.com/media/9xuUhwozi3qvJdPogI/giphy.gif" width=400 />
</center>

Let's test!

In [ ]:
fakedata = fakeData(hist, 100000, (-40,40))
hist = np.histogram(fakedata, bins=hist[0].size)
plt.hist(fakedata, density=True, bins=50)

Let's *compare*:

In [ ]:
hist_in = hist
fakedata = fakeData(hist, 100000, (-40,40))
hist_out = np.histogram(fakedata, bins=hist[0].size)
plt.plot(hist_in[0] * max(hist_out[0]) / max(hist_in[0]), label='input histogram')
plt.plot(hist_out[0], label='output histogram')

Wow!

Let's compare with professor's solution:

In [ ]:
hist[0]

In [ ]:
def fake_data_professor(A, N, R):
    my_random_variates = []
    x = np.linspace(R[0], R[1], len(A[0]))
    for _ in range(N):
        i = roulette_selection(A[0])
        if i == len(A[0]) - 1: i -= 1
        my_random_variates.append(random.uniform(x[i], x[i+1]))
    return my_random_variates

hist_in = hist
fakedataprofessor = fake_data_professor(hist_in, 100000, (-40,40))   
hist_out = np.histogram(fakedataprofessor, bins=hist[0].size)
plt.plot(hist_in[0] * max(hist_out[0]) / max(hist_in[0]), label='input histogram')
plt.plot(hist_out[0], label='output histogram')

Yang Huang's solution is *faster* and matches the input histogram *better*!

But... did Yang test *all possible inputs*? What if the random variate dataset size is *less* than the length of the histogram?

For example, try:
- `hist = np.histogram(data, bins=100)` and `fakedata = fakeData(hist, 300, (-40,40))`
- `hist = np.histogram(data, bins=1000)` and `fakedata = fakeData(hist, 750, (-40,40))`

### *Homework*
Compare Yang's solution with professor's solution. How would you improve Yang's solution so that it is closer to the desired output?

In [ ]:
hist = np.histogram(data, bins=100)

hist_in = hist
fakedata = fakeData(hist, 300, (-40,40))
hist_out1 = np.histogram(fakedata, bins=hist[0].size)
plt.plot(hist_in[0] * max(hist_out1[0]) / max(hist_in[0]), label='input histogram')
plt.plot(hist_out1[0], label='output histogram')

In [ ]:
hist_in = hist
fakedataprofessor = fake_data_professor(hist_in, 3000, (-40,40))   
hist_out2 = np.histogram(fakedataprofessor, bins=hist[0].size)
plt.plot(hist_in[0] * max(hist_out2[0]) / max(hist_in[0]), label='input histogram')
plt.plot(hist_out2[0], label='output histogram')

## 3. Random variates from histogram

Ok, we wrote the *prototype*, but now we're ready to write a general class that leverages key probability concepts such as **pdf** and **cdf**, does some unit testing, etc.

The following class will generate random variates, given a histogram. Note how I reduced tab spaces, long notebook cells are prettier that way.

In [ ]:
import pylab
import numpy

class GeneralRandom:
  """This class enables us to generate random numbers with an arbitrary 
  distribution."""
  
  def __init__(self, x = pylab.arange(-1.0, 1.0, .01), p = None, Nrl = 1000):
    """Initialize the lookup table (with default values if necessary)
    Inputs:
    x = random number values
    p = probability density profile at that point
    Nrl = number of reverse look up values between 0 and 1"""  
    
    if not isinstance(p, numpy.ndarray):
        if not isinstance(p, tuple):
            if p == None:
                p = pylab.exp(-10*x**2.0)
    self.set_pdf(x, p, Nrl)
   

  def set_pdf(self, x, p, Nrl = 1000):
    """Generate the lookup tables. 
    x is the value of the random variate
    pdf is its probability density
    cdf is the cumulative pdf
    inversecdf is the inverse look up table"""
    
    self.x = x
    if isinstance(p, tuple):
        self.pdf = p/sum(p) #normalize it
    else:
        self.pdf = p/p.sum() #normalize it
        
    self.cdf = self.pdf.cumsum()
    self.inversecdfbins = Nrl
    self.Nrl = Nrl
    
    y = pylab.arange(Nrl)/float(Nrl)
    delta = 1.0/Nrl
    self.inversecdf = pylab.zeros(Nrl)    
    self.inversecdf[0] = self.x[0]
    cdf_idx = 0
    
    for n in range(1,self.inversecdfbins):
        while self.cdf[cdf_idx] < y[n] and cdf_idx < Nrl:
            cdf_idx += 1
        self.inversecdf[n] = self.x[cdf_idx-1] + (
            self.x[cdf_idx] - self.x[cdf_idx-1]) * (y[n] - self.cdf[cdf_idx-1])/(self.cdf[cdf_idx] - self.cdf[cdf_idx-1]) 
        if cdf_idx >= Nrl:
            break
    self.delta_inversecdf = pylab.concatenate((pylab.diff(self.inversecdf), [0]))
              
  def random(self, N = 1000):
    """Give us N random numbers with the requested distribution"""

    idx_f = numpy.random.uniform(size = N, high = self.Nrl-1)
    idx = pylab.array([idx_f],'i')
    y = self.inversecdf[idx] + (idx_f - idx)*self.delta_inversecdf[idx]

    return y
  
  def plot_pdf(self):
    pylab.plot(self.x, self.pdf)
    
  def self_test(self, N = 1000):
    pylab.figure()
    #The cdf
    pylab.subplot(2,2,1)
    pylab.plot(self.x, self.cdf)
    #The inverse cdf
    pylab.subplot(2,2,2)
    y = pylab.arange(self.Nrl)/float(self.Nrl)
    pylab.plot(y, self.inversecdf)
    
    #The actual generated numbers
    pylab.subplot(2,2,3)
    y = self.random(N)
    p1, edges = pylab.histogram(y, bins = 50, 
                                range = (self.x.min(), self.x.max()), 
                                normed = True)
    x1 = 0.5*(edges[0:-1] + edges[1:])
    pylab.plot(x1, p1/p1.max())
    pylab.plot(self.x, self.pdf/self.pdf.max())

In [ ]:
test = GeneralRandom()
test.self_test()

In [ ]:
x = pylab.arange(-1.0, 1.0, .01)
pylab.exp(-10*x**2.0)

In [ ]:
type(data)

In [ ]:
data

This is our histogram from above: generated through random variates of a normal distribution:

In [ ]:
hist[0]

In [ ]:
type(hist_dist)

In [ ]:
hist_dist

So we have a histogram. Let's instantiate the class from that histogram in order to generate simulated data.

In [ ]:
rvs_from_hist = GeneralRandom(x = pylab.arange(-1.0, 1.0, .02), p = hist[0], Nrl = 100)
rvs_from_hist.self_test()

Now let's generate random variates:

In [ ]:
sim_data = rvs_from_hist.random()
sim_data

In [ ]:
sim_data[0]

In [ ]:
len(sim_data[0])

What does the data look like?

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.title("data simulated from given general distribution")
plt.plot(sim_data[0], label='sim_data')
plt.show()

Let's evaluate the histogram from the data, and also use `scipy.stats.rv_histogram` to generate random variates from that histogram:

In [ ]:
hist2 = np.histogram(sim_data, bins=100)
hist2_dist = scipy.stats.rv_histogram(hist2)

Let's plot the histogram of the random variates on top of the pdf from random variates from of the histogram of the random variates:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
X = np.linspace(-1.0, 1.0, 100)
plt.title("histogram and PDF from fake data")
plt.hist(sim_data[0], density=True, bins=100)
plt.plot(X, hist2_dist.pdf(X), label='PDF')
plt.show()

Let's do it again. Now we use the histogram above as the input to our class.

In [ ]:
rvs_from_hist2 = GeneralRandom(x = pylab.arange(-1.0, 1.0, .02), p = hist2[0], Nrl = 100)
rvs_from_hist2.self_test()

In [ ]:
sim_data2 = rvs_from_hist2.random()
sim_data2

Notice how the histograms degenerate in shape as we continue the process of generating numbers from a histogram, then using the generated numbers to get the new histogram, and using that as input to the same process.

In [ ]:
hist3 = np.histogram(sim_data2, bins=100)
hist3_dist = scipy.stats.rv_histogram(hist3)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
X = np.linspace(-1.0, 1.0, 100)
plt.title("histogram and PDF from fake data")
plt.hist(sim_data2[0], density=True, bins=100)
plt.plot(X, hist3_dist.pdf(X), label='PDF')
plt.show()

## 4. Empirical data

Now, we'll use empirical rather than simulated data: Tennesse rainfall.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
sns.set_context('notebook')

In [ ]:
precip = pd.read_table("data/nashville_precip.txt", index_col=0, na_values='NA', delim_whitespace=True)
precip.head()

In [ ]:
_ = precip.hist(sharex=True, sharey=True, grid=False)
plt.tight_layout()

In [ ]:
precip.fillna(value={'Oct': precip.Oct.mean()}, inplace=True)

This is precipitation for the month of April and its histogram:

In [ ]:
precip.Apr

In [ ]:
april_h = precip.Apr.hist(normed=True, bins=30)

In [ ]:
precip.Apr.values

In [ ]:
wx_hist = np.histogram(precip.Apr.values, bins=100)
wx_hist

In [ ]:
wx_hist[0]

Let's use the histogram from precipitation for the month of April as input. First, we'll do it with our prototype, then with the general class we wrote:

In [ ]:
precip.Apr.values.min()

In [ ]:
precip.Apr.values.max()

In [ ]:
# inputs
h = wx_hist[0]
r = (precip.Apr.values.min(), precip.Apr.values.max())
n = 10000

# we build
x = np.linspace(r[0], r[1], len(h))
x[0], x[len(h)//2], x[len(h)-1]

In [ ]:
my_random_variates = []
for _ in range(n):
    i = roulette_selection(h)
    if i == len(h) - 1: i -= 1
    my_random_variates.append(random.uniform(x[i], x[i+1]))
    
my_not_so_random_histogram = np.histogram(my_random_variates, bins=len(h))

plt.title("my not-so-random histogram")
plt.plot(my_not_so_random_histogram[0], label='generated histogram')
plt.plot(my_not_so_random_histogram[0].max() / h.max() * h, label='original histogram')
plt.show()

</br >
<center>
<img src="https://media3.giphy.com/media/7NOL7wCK1ddCQTiv51/giphy.gif" width=400 />
</center>

Now let's use our class:

In [ ]:
wx_from_hist = GeneralRandom(x = pylab.arange(-1.0, 1.0, .02), p = wx_hist[0], Nrl = 100)
wx_from_hist.self_test()

Let's generate random precipitation values from the month of April that abide by that histogram:

In [ ]:
wx_sim_data = wx_from_hist.random()
wx_sim_data

In [ ]:
wx_sim_data[0]

Let's plot the histogram of these fake precipitation values:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.title("histogram from fake wx data")
plt.plot(np.histogram(wx_sim_data[0], bins=100)[0], label='histogram')
plt.show()

Does the histogram from the fake precipitation values look like Tennessee's real April histogram?

In [ ]:
april_h = precip.Apr.hist(normed=True, bins=30)

# Conclusion

Math is by far the best model, since functions give us the *lowest possible model dimensionality reduction*. But when the math is too tough, you can always pick a *program*. The number of SLOC in the program will be much bigger than the number of parameters in your analytic function, but if your model *rocks*, like the one in this notebook, then use it!

That is the principle behind Machine Leanring (ML). Sometimes, the math is too tough for very strange-looking (or lots of) data. So instead of building a model with math functions, we build a model by writing a program that matches the statistics of our input dataset + label. That is what Artificial Neural Networks do (because graphs are a general way of drawing curves in n dimensions) and what deep learning is all about, and we'll see this when you come back fro spring break (after we study linear algebra and graphs).